In [1]:
import os

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib.patches as patches
import cv2

%matplotlib inline


In [3]:
from Datasets.dataset import Dataset

dataset = Dataset(path="./Datasets/OTB50/", test_split="Display.txt", train_split="Display.txt")
frames, annots = dataset.get_next_video_frames_and_annots(test=True)

Loading next video
New Epoch:  1
Loaded next train video:  Biker
Loading next video
Loaded next test video:  Biker
Loading next video
Loaded next test video:  Biker


In [4]:
def animate(frames, preds, gt_annots=None):
    fig, ax = plt.subplots(1)
    ims = []
    
    ndots = preds[0].shape[-1]//2
    scaler = np.tile(np.array([frames[0].shape[1], frames[0].shape[0]]), [ndots])
    
    for i,(frame, anot) in enumerate(zip(frames, preds)):
            
        s_anot = (anot*scaler).astype(int)     
        cv2.rectangle(frame,(s_anot[0],s_anot[1]),(s_anot[0]+s_anot[2],s_anot[1]+s_anot[3]), (0,255,0),3)
        
        if gt_annots is not None:
            s_gt_anot = (gt_annots[i]*scaler).astype(int)
            cv2.rectangle(frame,(s_gt_anot[0],s_gt_anot[1]),(s_gt_anot[0]+s_gt_anot[2],s_gt_anot[1]+s_gt_anot[3]), (0,0,255),3)
            
        im = plt.imshow(frame, animated=True)

        ims.append([im])
        
    ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True, repeat=False)
    plt.close()
    
    return HTML(ani.to_html5_video())

In [5]:
# html_vid = animate(frames, annots, gt=True)
# display(html_vid)

In [6]:
from Animator import Animator
from RLT_model import RLT

In [7]:
experiment_name = "DRLT_long"
time_steps = 10
max_time_steps = 1500  # longest video in the dataset (girl)
batch_size = 5
model = RLT(max_time_steps=max_time_steps, time_steps=time_steps, batch_size=batch_size)
animator = Animator(dataset, model, batch_size=batch_size, time_steps=time_steps, max_time_steps=max_time_steps, experiment_name=experiment_name)

det_actions = animator.get_det_actions(frames, annots)

ll: 55-leaky
checkpoint_dir: /home/gkovac/Seminar/RL_Tracking/experiments/DRLT_long/checkpoint
Loading
INFO:tensorflow:Restoring parameters from /home/gkovac/Seminar/RL_Tracking/experiments/DRLT_long/checkpoint/DRLT_long-410476


In [8]:
html_vid = animate(frames, det_actions, annots)
display(html_vid)